In [41]:
from sklearn.utils import shuffle
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
pd.options.mode.chained_assignment = None  # default='warn'

In [42]:
file_a = r".\raw_data\scopus\scopus_2000.csv"
file_g = r".\raw_data\scopus\scopus_22_23.csv"
file_r = r".\raw_data\scopus\scopus_19_20_21.csv"

df_g = pd.read_csv(file_g)
df_a = pd.read_csv(file_a)
df_r = pd.read_csv(file_r)

df = pd.concat([df_a, df_g, df_r], axis=0)
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.fillna("blank")

In [43]:
df

,Authors,Title,Year,Source title,DOI,Link,Abstract,Author Keywords,Index Keywords,Document Type,Source
0,"Manjula A., Narsimha G.",XCYPF: A flexible and extensible framework for...,2015,Proceedings of 2015 IEEE 9th International Con...,10.1109/ISCO.2015.7282311,https://www.scopus.com/inward/record.uri?eid=2...,Precision agriculture is the technology driven...,Crop yield prediction; Data mining techniques;...,Atmospheric temperature; Crops; Data mining; F...,Conference Paper,Scopus
1,"Manimekalai S., Nandhini K.",Crop yield prediction from soil parameters thr...,2018,International Journal of Engineering and Techn...,blank,https://www.scopus.com/inward/record.uri?eid=2...,Agriculture is considered as the backbone of t...,Agriculture; Classification; Crop yield predic...,blank,Article,Scopus
2,"Gandge Y., Sandhya",A study on various data mining techniques for ...,2018,"International Conference on Electrical, Electr...",10.1109/ICEECCOT.2017.8284541,https://www.scopus.com/inward/record.uri?eid=2...,India is a country where agriculture and agric...,accuracy and Recommendation; crop yield predic...,Crops; Forecasting; Losses; Models; accuracy a...,Conference Paper,Scopus
3,"Ananthara M.G., Arunkumar T., Hemavathy R.",CRY-An improved crop yield prediction model us...,2013,Proceedings of the 2013 International Conferen...,10.1109/ICPRIME.2013.6496717,https://www.scopus.com/inward/record.uri?eid=2...,Agricultural researchers over the world insist...,Bee Hive clustering algorithm; Crop Growth; Cr...,Agricultural Data sets; Clustering approach; C...,Conference Paper,Scopus
4,"Gandhi N., Armstrong L.J., Petkar O.",Proposed decision support system (DSS) for Ind...,2016,Proceedings - 2016 IEEE International Conferen...,10.1109/TIAR.2016.7801205,https://www.scopus.com/inward/record.uri?eid=2...,Rice crop production provides more than 40% to...,crop yield prediction; decision support system...,Agriculture; Artificial intelligence; Crops; C...,Conference Paper,Scopus
...,...,...,...,...,...,...,...,...,...,...,...
4092,"Karthikeyan L., Chawla I., Mishra A.K.",A review of remote sensing applications in agr...,2020,Journal of Hydrology,10.1016/j.jhydrol.2020.124905,https://www.scopus.com/inward/record.uri?eid=2...,The global population is expected to reach 9.8...,Agriculture; Crop losses; Crop yield; Data ass...,Agricultural robots; Agriculture; Climate chan...,Article,Scopus
4093,"Xu D., Li T., Li Y., Su X., Tarkoma S., Jiang ...",Edge Intelligence: Empowering Intelligence to ...,2021,Proceedings of the IEEE,10.1109/JPROC.2021.3119950,https://www.scopus.com/inward/record.uri?eid=2...,Edge intelligence refers to a set of connected...,Artificial intelligence (AI); edge caching; ed...,Artificial intelligence; Data handling; Survey...,Article,Scopus
4094,"Zenda T., Liu S., Dong A., Duan H.",Advances in cereal crop genomics for resilienc...,2021,Life,10.3390/life11060502,https://www.scopus.com/inward/record.uri?eid=2...,"Adapting to climate change, providing sufficie...",Cereal crops; Crop phenotyping; Crop wild rela...,blank,Review,Scopus
4095,"Mouazen A.M., Alexandridis T., Buddenbaum H., ...",Monitoring,2020,Agricultural Internet of Things and Decision S...,10.1016/B978-0-12-818373-1.00002-0,https://www.scopus.com/inward/record.uri?eid=2...,The first requirement for successful implement...,Crop; High sampling resolution; Proximal sensi...,blank,Book Chapter,Scopus


In [44]:
df['Abstract'] = df['Abstract'].str.lower()
df['Title'] = df['Title'].str.lower()
df['Author Keywords'] = df['Author Keywords'].str.lower()
df['Index Keywords'] = df['Index Keywords'].str.lower()

In [45]:
df.head(3)

,Authors,Title,Year,Source title,DOI,Link,Abstract,Author Keywords,Index Keywords,Document Type,Source
0,"Manjula A., Narsimha G.",xcypf: a flexible and extensible framework for...,2015,Proceedings of 2015 IEEE 9th International Con...,10.1109/ISCO.2015.7282311,https://www.scopus.com/inward/record.uri?eid=2...,precision agriculture is the technology driven...,crop yield prediction; data mining techniques;...,atmospheric temperature; crops; data mining; f...,Conference Paper,Scopus
1,"Manimekalai S., Nandhini K.",crop yield prediction from soil parameters thr...,2018,International Journal of Engineering and Techn...,blank,https://www.scopus.com/inward/record.uri?eid=2...,agriculture is considered as the backbone of t...,agriculture; classification; crop yield predic...,blank,Article,Scopus
2,"Gandge Y., Sandhya",a study on various data mining techniques for ...,2018,"International Conference on Electrical, Electr...",10.1109/ICEECCOT.2017.8284541,https://www.scopus.com/inward/record.uri?eid=2...,india is a country where agriculture and agric...,accuracy and recommendation; crop yield predic...,crops; forecasting; losses; models; accuracy a...,Conference Paper,Scopus


In [46]:
non_alp = [';', ',', '(', ')', '[', ']', '.']
for i in non_alp:
    for col in ['Title','Abstract', 'Author Keywords', 'Index Keywords']:
        df[col] = df[col].map(lambda x: x.replace(i,''))

In [47]:
stop = stopwords.words('english')
for i in [df['Abstract'],df['Title'], df['Author Keywords'], df['Index Keywords']]:
    i = i.apply(lambda x: " ".join([word for word in x.split() if word not in (stop)]))

In [53]:
DIC = {"AI": ['Machine intelligence', 'Cognitive computing', 'Automated reasoning', 'Intelligent automation', 'Intelligent machines', 
            'Smart technology', 'Robotics', 'Expert systems', 'Neural networks', 'Deep learning', 'Machine learning', 
            'Natural language processing', 'Computer vision', 'Autonomous systems', 'Intelligent agents', 'Intelligent robotics', 
            'Cognitive AI', 'Smart systems', 'Neural computing', 'Intelligent algorithms', 'Advanced analytics', 'Augmented intelligence', 
            'Automated intelligence', 'Intelligent software', 'Intelligent systems', 'Intelligent machines', 'Smart devices', 
            'Cognitive automation', 'Machine perception', 'Intelligent computing', 'Predictive analytics', 'Computational intelligence', 
            'Machine reasoning', 'Smart machines', 'Intelligent computing systems', 'Intelligent assistants', 'Smart robots', 
            'Cognitive robotics', 'Intelligent sensors', 'Intelligent decision making',"ANN", "CNN", "Backpropagation networks", "DNN", "DBN", "RNN"],
        "XAI": ['Transparent artificial intelligence', 'Interpretable artificial intelligence', 'Explainable machine learning for artificial intelligence', 
            'Human-interpretable artificial intelligence', 'Intelligible artificial intelligence', 'Understandable artificial intelligence', 
            'Clear box artificial intelligence', 'Accountable artificial intelligence', 'Trustworthy artificial intelligence', 
            'Ethical artificial intelligence', 'Open box artificial intelligence', 'Auditable artificial intelligence', 
            'Scrutable artificial intelligence', 'Insightful artificial intelligence', 'Comprehensible artificial intelligence', 
            'Cognitively accessible artificial intelligence', 'Communicative artificial intelligence', 'Discernible artificial intelligence', 
            'Introspective artificial intelligence', 'Explainable decision making for artificial intelligence', 'Justifiable artificial intelligence', 
            'Rational artificial intelligence', 'Reasonable artificial intelligence', 'Legitimate artificial intelligence', 
            'Verified artificial intelligence', 'Validated artificial intelligence', 'Empathetic artificial intelligence', 
            'Engaging artificial intelligence', 'Informative artificial intelligence', 'Intervenable artificial intelligence', "XAI", "XML"]}




In [52]:
weights = {'Title': 0.4, 'Author Keywords': 0.3, 'Index Keywords': 0.2, 'Abstract':0.1}

# Create a function to count the number of times the words appear in a row
def count_words(row):
    counts = {}
    for key in list(DIC.keys()):
        counts[key] = 0
        for word in DIC[key]:
            for col in row.index:
                if col not in ['Authors', 'DOI', 'Link'] and word.lower() in re.findall(r'\b\w+\b',row[col]):
                    count = counts.get(key, 0)
                    counts[key] = count + weights.get(col, 1) if count > 0 else weights.get(col, 1)
    max_word = max(counts, key=counts.get) if any(counts.values()) else np.nan                    
    return pd.Series(counts), max_word

# Apply the function to each row of the dataframe
word_counts, max_words = zip(*df.apply(count_words, axis=1))

# Create a new dataframe with the word counts as columns
words_count_df = pd.DataFrame(list(word_counts), columns=list(DIC.keys()))

# Add the max_word column to the original dataframe
df['Category'] = max_words

# Concatenate the two dataframes
result_df = pd.concat([df, words_count_df], axis=1)
result_df

TypeError: expected string or bytes-like object

In [54]:
for key in DIC.keys():
    df[key] = 0
    for phr in DIC[key]:
        for col in [df['Abstract'], df['Title'], df['Author Keywords'], df['Index Keywords']]:
            if col is df['Title']:
                df[key] += col.str.count(phr.lower()) #+ 0.0004
            elif col is df['Author Keywords']:
                #df[key] += 0.00003
                df[key] += col.str.count(phr.lower()) #+ 0.0003
            elif col is df['Index Keywords']:
                #df[key] += 0.00002
                df[key] += col.str.count(phr.lower())  #+ 0.0002
            else:
                #df[key] += 0.00001
                df[key] += col.str.count(phr.lower()) #+ 0.0001

In [56]:
df[df["XAI"] > 0]

,Authors,Title,Year,Source title,DOI,Link,Abstract,Author Keywords,Index Keywords,Document Type,Source,AI,XAI
1144,"Ahmed M.S., Tazwar M.T., Khan H., Roy S., Iqba...",yield response of different rice ecotypes to m...,2022,Complexity,10.1155/2022/5305353,https://www.scopus.com/inward/record.uri?eid=2...,the food security of more than half of the wor...,blank,adaptive boosting food supply forecasting lime...,Article,Scopus,4,1
1772,"Ravi M., Negi A., Chitnis S.",a comparative review of expert systems recomme...,2022,2022 IEEE 7th International conference for Con...,10.1109/I2CT54291.2022.9824265,https://www.scopus.com/inward/record.uri?eid=2...,previously expert systems es dominated artific...,and explainable artificial intelligence xai ar...,deep learning expert systems learning systems ...,Conference Paper,Scopus,11,4
1913,"Chakraborty D., Başağaoğlu H., Alian S., Mirch...",multiscale extrapolative learning algorithm fo...,2023,Expert Systems with Applications,10.1016/j.eswa.2022.119056,https://www.scopus.com/inward/record.uri?eid=2...,we present multiscale extrapolative learning a...,artificial intelligence climate change impacts...,arid regions artificial intelligence climate c...,Article,Scopus,1,3
2515,[No author name available],5th international conference on information an...,2021,Lecture Notes in Networks and Systems,blank,https://www.scopus.com/inward/record.uri?eid=2...,the proceedings contain 106 papers the special...,blank,blank,Conference Review,Scopus,4,1
2895,"Wolanin A., Mateo-Garciá G., Camps-Valls G., G...",estimating and understanding crop yields with ...,2020,Environmental Research Letters,10.1088/1748-9326/ab68ac,https://www.scopus.com/inward/record.uri?eid=2...,forecasting crop yields is becoming increasing...,deep learning dl explainable artificial intell...,chemical activation climate change crops decis...,Article,Scopus,8,2
3329,Khan M.A.,towards efficient xml parsing through minimiza...,2019,Journal of Supercomputing,10.1007/s11227-018-2721-y,https://www.scopus.com/inward/record.uri?eid=2...,a significant increase in the usage of extensi...,machine learning parameter space exploration p...,computational linguistics hypertext systems ja...,Article,Scopus,4,11
